In [ ]:
from lppls import lppls
import numpy as np
import pandas as pd
from datetime import datetime as dt

import os

%matplotlib inline

def read_data():
    local_path = os.path.abspath('')
    data_path = os.path.join(local_path, 'lppls-extras', os.environ['SAFE_TICKER']+'.csv')
    return pd.read_csv(data_path, encoding='utf-8')

data = read_data()
time = [pd.Timestamp.toordinal(dt.strptime(t1, '%Y-%m-%d')) for t1 in data['datetime']]
price = np.log(data['close'].values)
observations = np.array([time, price])

# Tested:
# lppls_q.QLPPLS(observations=observations) # isn't as accurate for more volitle assets
lppls_model = lppls.LPPLS(observations=observations)

# set the max number for searches to perform before giving-up
# the literature suggests 25
#
# Tested:
# MAX_SEARCHES = 50 # doesn't seem to do much
MAX_SEARCHES = 25

# fit the model to the data and get back the params
tc, m, w, a, b, c, c1, c2, O, D = lppls_model.fit(MAX_SEARCHES)

# visualize the fit
lppls_model.plot_fit()

# compute the confidence indicator
res = lppls_model.mp_compute_nested_fits(
    workers=8,
    # args from the readme
    # Tested (default args):
    # window_size=80, smallest_window_size=20, outer_increment=5, inner_increment=2, max_searches=25, # faster, but not as much info
    window_size=120,
    smallest_window_size=30,
    outer_increment=1,
    inner_increment=5,
    max_searches=25,
    filter_conditions_config={},
)
lppls_model.plot_confidence_indicators(res)